# Known bugs - generated parameters are not being passed to local tools

In [1]:
!pip install xpander-sdk llama-index llama-index-llms-openai python-dotenv


  Using cached llama_index-0.12.15-py3-none-any.whl.metadata (12 kB)
  Using cached llama_index_llms_openai-0.3.15-py3-none-any.whl.metadata (3.3 kB)
  Using cached llama_index_agent_openai-0.4.3-py3-none-any.whl.metadata (727 bytes)
  Using cached llama_index_cli-0.4.0-py3-none-any.whl.metadata (1.5 kB)
  Using cached llama_index_core-0.12.15-py3-none-any.whl.metadata (2.5 kB)
  Using cached llama_index_embeddings_openai-0.3.1-py3-none-any.whl.metadata (684 bytes)
  Using cached llama_index_indices_managed_llama_cloud-0.6.4-py3-none-any.whl.metadata (3.6 kB)
  Using cached llama_index_multi_modal_llms_openai-0.4.3-py3-none-any.whl.metadata (726 bytes)
  Using cached llama_index_program_openai-0.3.1-py3-none-any.whl.metadata (764 bytes)
  Using cached llama_index_question_gen_openai-0.3.0-py3-none-any.whl.metadata (783 bytes)
  Using cached llama_index_readers_file-0.4.4-py3-none-any.whl.metadata (5.4 kB)
  Using cached llama_index_readers_llama_parse-0.4.0-py3-none-any.whl.metadata (3

In [2]:
from llama_index.core.agent import ReActAgent
from llama_index.llms.openai import OpenAI
from llama_index.core.tools import FunctionTool
from os import environ
from dotenv import load_dotenv

load_dotenv()

OPENAI_API_KEY = environ["OPENAI_API_KEY"]
XPANDER_API_KEY = environ["XPANDER_API_KEY"]
XPANDER_AGENT_ID = environ["XPANDER_AGENT_ID"]

In [3]:
def multiply(a: int, b: int) -> int:
    """Multiply two integers and returns the result integer"""
    return a * b


multiply_tool = FunctionTool.from_defaults(fn=multiply)

def add(a: int, b: int) -> int:
    """Add two integers and returns the result integer"""
    return a + b


add_tool = FunctionTool.from_defaults(fn=add)

def square(a: int) -> int:
    """Multiply a by itself and returns the result integer"""
    return a*a

square_tool = FunctionTool.from_defaults(fn=square)

In [4]:
# llm = OpenAI(model="gpt-3.5-turbo")
llm = OpenAI(model="gpt-4")
agent = ReActAgent.from_tools([multiply_tool, add_tool, square_tool], llm=llm, verbose=True)

In [5]:
response_gen = agent.stream_chat("What is 20+2*4? Calculate step by step")
response_gen.print_response_stream()

> Running step 9f589385-c79d-46fb-8e03-670106b835fb. Step input: What is 20+2*4? Calculate step by step
Thought: The user wants to calculate the expression 20+2*4 step by step. According to the order of operations (BIDMAS/BODMAS), multiplication should be performed before addition. So, I will first multiply 2 and 4, then add the result to 20.
Action: multiply
Action Input: {'a': 2, 'b': 4}
Observation: 8
> Running step 7c789aac-dfd2-4df0-96cd-a04ebdaf2265. Step input: None
Thought: The multiplication of 2 and 4 is 8. Now, I will add this result to 20.
Action: add
Action Input: {'a': 20, 'b': 8}
Observation: 28
> Running step 345e2673-b565-4bc5-bb7e-14438e46fc80. Step input: None
 The result of the calculation 20+2*4 is 28.

In [6]:
response_gen_3 = agent.stream_chat("What is the square of 14 + 12 squared? Calculate step by step")

> Running step c35989d5-5c35-4854-bc17-b47f5d3f84f5. Step input: What is the square of 14 + 12 squared? Calculate step by step
Thought: The user wants to calculate the square of 14 and the square of 12, then add them together. I need to use the 'square' tool twice, once for each number, and then the 'add' tool to add the results together.
Action: square
Action Input: {'a': 14}
Observation: 196
> Running step ff259cde-54a6-4f28-8341-47a758262eb6. Step input: None
Thought: The square of 14 is 196. Now I need to calculate the square of 12.
Action: square
Action Input: {'a': 12}
Observation: 144
> Running step 485bda74-f42d-42ee-bce0-e4e5fc4d86ce. Step input: None
Thought: The square of 12 is 144. Now I need to add 196 and 144 together.
Action: add
Action Input: {'a': 196, 'b': 144}
Observation: 340
> Running step bbbc49b2-3d49-4bb7-969f-7409720540a6. Step input: None


In [7]:
response_gen_3.print_response_stream()

 The square of 14 plus the square of 12 equals 340.

In [8]:
response_gen_4 = agent.stream_chat("What is the square of 14 + the square of 12? Calculate step by step")
response_gen_4.print_response_stream()

> Running step 224e55c8-f816-499f-a4a3-545af51ff002. Step input: What is the square of 14 + the square of 12? Calculate step by step
Thought: The user wants to know the result of the square of 14 added to the square of 12. I will use the 'square' tool twice to get the squares of 14 and 12, and then use the 'add' tool to add the results.
Action: square
Action Input: {'a': 14}
Observation: 196
> Running step 4ecae98c-6b0b-427a-aeba-7953821bc1a0. Step input: None
Thought: The square of 14 is 196. Now I need to find the square of 12.
Action: square
Action Input: {'a': 12}
Observation: 144
> Running step 3610518c-a0d3-4f0a-9075-9ac460bc9c5f. Step input: None
Thought: The square of 12 is 144. Now I need to add 196 and 144.
Action: add
Action Input: {'a': 196, 'b': 144}
Observation: 340
> Running step e5897bda-9da8-4601-b5d4-d615f115fb8c. Step input: None
 The square of 14 plus the square of 12 equals 340.

In [9]:
from openai import OpenAI
from xpander_sdk import XpanderClient, LLMProvider
# load the client
xpander_client = XpanderClient(api_key=XPANDER_API_KEY)

# get the agent
xpander_agent = xpander_client.agents.get(agent_id=XPANDER_AGENT_ID)
openai_client = OpenAI(api_key=OPENAI_API_KEY)

b'{"stdout":"bG9hZGluZyBhZ2VudCBkY2NhY2I0Ny1mOTBmLTRlOTYtYmYyZi1hOTI1ODZiZGM2NDEK"}\n'


In [10]:
local_tools = [
    {
        "type": "function",
        "function": {
            "name": "multiply",
            "description": "Multiply two integers and returns the result integer",
            "parameters": {
                "type": "object",
                "properties": {
                    "a": {
                        "type": "integer",
                        "description": "First integer to multiply"
                    },
                    "b": {
                        "type": "integer",
                        "description": "Second integer to multiply"
                    }
                },
                "required": ["a", "b"]
            }
        }
    },
    {
        "type": "function",
        "function": {
            "name": "add",
            "description": "Add two integers and returns the result integer",
            "parameters": {
                "type": "object",
                "properties": {
                    "a": {
                        "type": "integer",
                        "description": "First integer to add"
                    },
                    "b": {
                        "type": "integer",
                        "description": "Second integer to add"
                    }
                },
                "required": ["a", "b"]
            }
        }
    },
    {
        "type": "function",
        "function": {
            "name": "square",
            "description": "Multiply a by itself and returns the result integer",
            "parameters": {
                "type": "object",
                "properties": {
                    "a": {
                        "type": "integer",
                        "description": "Integer to square"
                    }
                },
                "required": ["a"]
            }
        }
    }
]


xpander_agent.add_local_tools(local_tools)
xpander_agent.get_tools()

[{'type': 'function',
  'function': {'name': 'xpLocal_multiply',
   'description': 'Multiply two integers and returns the result integer',
   'parameters': {'type': 'object',
    'properties': {'a': {'type': 'integer',
      'description': 'First integer to multiply'},
     'b': {'type': 'integer', 'description': 'Second integer to multiply'}},
    'required': ['a', 'b']}}},
 {'type': 'function',
  'function': {'name': 'xpLocal_add',
   'description': 'Add two integers and returns the result integer',
   'parameters': {'type': 'object',
    'properties': {'a': {'type': 'integer',
      'description': 'First integer to add'},
     'b': {'type': 'integer', 'description': 'Second integer to add'}},
    'required': ['a', 'b']}}},
 {'type': 'function',
  'function': {'name': 'xpLocal_square',
   'description': 'Multiply a by itself and returns the result integer',
   'parameters': {'type': 'object',
    'properties': {'a': {'type': 'integer',
      'description': 'Integer to square'}},
    

In [11]:

def handle_local_tool_calls(tool_call):
    """Handle local tool calls for calculator functions"""
    result = None
    params = tool_call.payload.get("bodyParams", {})
    
    if tool_call.name == "multiply":
        result = multiply(
            params.get("a"),
            params.get("b")
        )
    elif tool_call.name == "add":
        result = add(
            params.get("a"),
            params.get("b")
        )
    elif tool_call.name == "square":
        result = square(
            params.get("a")
        )
    
    return result

In [12]:
from xpander_sdk import ToolCallType
# create execution

xpander_agent.add_task("What is the square of 14 + the square of 12? Calculate step by step")

while not xpander_agent.is_finished():
    print("-"*100)
    response = openai_client.chat.completions.create(
                model= 'gpt-4o',
                messages=xpander_agent.memory.retrieve_messages(),
                tools=xpander_agent.get_tools(llm_provider=LLMProvider.OPEN_AI),
                tool_choice="auto",
                temperature=0.0
        )
    print(response.model_dump())
    # add messages directly from the LLM response
    xpander_agent.memory.add_messages(response.model_dump())
    
    # extract tool calls from the LLM response
    tool_calls = XpanderClient.extract_tool_calls(llm_response=response.model_dump(),llm_provider=LLMProvider.OPEN_AI)
    # run tools
    for tool_call in tool_calls:
        if tool_call.type == ToolCallType.LOCAL:
            result = handle_local_tool_calls(tool_call)
            xpander_agent.add_tool_result(tool_call=tool_call, result=result)
        else:
            xpander_agent.run_tool(tool_call=tool_call)
    
print(xpander_agent.retrieve_execution_result().result)

----------------------------------------------------------------------------------------------------
{'id': 'chatcmpl-AvtHWsz33toLuf7SySJ9PrnxQbxmb', 'choices': [{'finish_reason': 'tool_calls', 'index': 0, 'logprobs': None, 'message': {'content': None, 'refusal': None, 'role': 'assistant', 'audio': None, 'function_call': None, 'tool_calls': [{'id': 'call_H2lnt1TpJEbN6AQYt5In2CY0', 'function': {'arguments': '{"a": 14}', 'name': 'xpLocal_square'}, 'type': 'function'}, {'id': 'call_XiEf1pdZEuOiMedUqG7w9gRT', 'function': {'arguments': '{"a": 12}', 'name': 'xpLocal_square'}, 'type': 'function'}]}}], 'created': 1738360562, 'model': 'gpt-4o-2024-08-06', 'object': 'chat.completion', 'service_tier': 'default', 'system_fingerprint': 'fp_50cad350e4', 'usage': {'completion_tokens': 49, 'prompt_tokens': 2327, 'total_tokens': 2376, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audi

TypeError: unsupported operand type(s) for *: 'NoneType' and 'NoneType'